In [29]:
# Install required packages
!pip install flask-ngrok openpyxl lime scikit-learn --quiet

In [27]:
import pandas as pd
import pickle
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# === STEP 1: Load Dataset ===
train_file = "/content/credit_score_training_data.xlsx"
df = pd.read_excel(train_file)

# Encode categorical features
label_encoders = {}
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Split into features and target
X = df.drop(columns=['CREDIT_SCORE'])
y = df['CREDIT_SCORE']

# === STEP 2: Define Models to Compare ===
models_to_try = {
    "RandomForest": lambda seed: RandomForestRegressor(random_state=seed),
    "GradientBoosting": lambda seed: GradientBoostingRegressor(random_state=seed),
    "MLPRegressor": lambda seed: MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=seed)
}

# === STEP 3: Train and Evaluate Models ===
best_r2 = float('-inf')
best_model = None
best_model_name = ""
best_scaler = None
best_comparison_df = None
best_epoch = None

for model_name, model_func in models_to_try.items():
    print(f"\nTraining model: {model_name}")

    for epoch in range(1, 21):
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=epoch
        )

        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Initialize and train model
        model = model_func(epoch)
        model.fit(X_train_scaled, y_train)

        # Predict
        y_pred = model.predict(X_test_scaled)

        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        print(f"Epoch {epoch:2d} | R²: {r2:.4f} | RMSE: {rmse:.2f} | MAE: {mae:.2f}")

        # Track best model overall
        if r2 > best_r2:
            best_r2 = r2
            best_model = model
            best_model_name = model_name
            best_scaler = scaler
            best_epoch = epoch
            best_comparison_df = pd.DataFrame({
                'Actual': y_test.values,
                'Predicted': y_pred.round(2)
            })

# === STEP 4: Save the Best Model ===
with open("credit_score_model.pkl", "wb") as f:
    pickle.dump(best_model, f)
with open("scaler.pkl", "wb") as f:
    pickle.dump(best_scaler, f)
with open("encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

# === STEP 5: Summary ===
print("\nBest model saved successfully.")
print(f"Best Model: {best_model_name} from epoch {best_epoch}")
print(f"Best R² Score: {best_r2:.4f}")
print("\nSample Prediction Results:")
print(best_comparison_df.head(10))



Training model: RandomForest
Epoch  1 | R²: 0.5964 | RMSE: 38.12 | MAE: 27.30
Epoch  2 | R²: -0.6000 | RMSE: 18.97 | MAE: 15.60
Epoch  3 | R²: -1.2059 | RMSE: 14.85 | MAE: 14.85
Epoch  4 | R²: 0.7144 | RMSE: 42.75 | MAE: 34.35
Epoch  5 | R²: 0.7739 | RMSE: 38.04 | MAE: 32.30
Epoch  6 | R²: -1.9266 | RMSE: 42.77 | MAE: 33.45
Epoch  7 | R²: -5.1943 | RMSE: 49.78 | MAE: 41.85
Epoch  8 | R²: 0.1008 | RMSE: 47.41 | MAE: 39.55
Epoch  9 | R²: -30.7743 | RMSE: 56.37 | MAE: 52.45
Epoch 10 | R²: -0.0600 | RMSE: 41.18 | MAE: 30.20
Epoch 11 | R²: -1.1464 | RMSE: 51.28 | MAE: 38.95
Epoch 12 | R²: -5.4163 | RMSE: 50.66 | MAE: 43.45
Epoch 13 | R²: 0.9728 | RMSE: 9.90 | MAE: 9.60
Epoch 14 | R²: -6.2340 | RMSE: 53.79 | MAE: 50.40
Epoch 15 | R²: -2.4694 | RMSE: 18.63 | MAE: 16.65
Epoch 16 | R²: -1.9510 | RMSE: 17.18 | MAE: 16.45
Epoch 17 | R²: 0.8990 | RMSE: 1.59 | MAE: 1.55
Epoch 18 | R²: 0.6723 | RMSE: 37.21 | MAE: 27.10
Epoch 19 | R²: 0.5774 | RMSE: 6.50 | MAE: 6.50
Epoch 20 | R²: 0.4080 | RMSE: 7.6

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Epoch  1 | R²: -51.8115 | RMSE: 436.03 | MAE: 425.65


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Epoch  2 | R²: -111.4801 | RMSE: 159.08 | MAE: 143.36


...

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Epoch 18 | R²: -53.4129 | RMSE: 479.47 | MAE: 479.28


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Epoch 19 | R²: -1383.1646 | RMSE: 372.04 | MAE: 363.04
Epoch 20 | R²: -1382.9552 | RMSE: 372.02 | MAE: 358.99

Best model saved successfully.
Best Model: GradientBoosting from epoch 13
Best R² Score: 0.9962

Sample Prediction Results:
   Actual  Predicted
0     640     642.48
1     760     755.37


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [28]:
from lime.lime_tabular import LimeTabularExplainer

test_file = "/content/TEST_DATA.xlsx"  # Replace with your path

try:
    input_df = pd.read_excel(test_file)

    required_cols = ['AGE', 'INCOME', 'EMPLOYMENT_STATUS', 'LOAN_AMOUNT', 'LOAN_TERM',
                     'INTEREST_RATE', 'NUM_OF_DEPENDENTS', 'MARITAL_STATUS', 'EDUCATION_LEVEL']

    if not set(required_cols).issubset(input_df.columns):
        print(f"Missing columns: {required_cols}")
    else:
        model = pickle.load(open("credit_score_model.pkl", "rb"))
        scaler = pickle.load(open("scaler.pkl", "rb"))
        label_encoders = pickle.load(open("encoders.pkl", "rb"))

        for col in input_df.select_dtypes(include=['object']).columns:
            if col in label_encoders:
                le = label_encoders[col]
                known_classes = set(le.classes_)
                input_df[col] = input_df[col].apply(lambda x: le.transform([x])[0] if x in known_classes else -1)
            else:
                input_df[col] = input_df[col].astype('category').cat.codes

        input_scaled = scaler.transform(input_df[required_cols])
        predictions = model.predict(input_scaled)

        # LIME Explainer Setup
        explainer = LimeTabularExplainer(
            training_data=X_train_scaled,
            feature_names=X.columns.tolist(),
            mode="regression"
        )

        risk_categories, decisions, explanations = [], [], []

        for i, score in enumerate(predictions):
            if score >= 750:
                risk = "Low Risk"
                decision = "Likely Loan Approval"
            elif score >= 650:
                risk = "Medium Risk"
                decision = "Possible Loan Approval with Caution"
            else:
                risk = "High Risk"
                decision = "Loan Likely Rejected or Needs Collateral"

            risk_categories.append(risk)
            decisions.append(decision)

            # Get LIME explanation
            exp = explainer.explain_instance(input_scaled[i], model.predict, num_features=5)
            explanation_sentences = []
            for feature, weight in exp.as_list():
                impact = "increased" if weight > 0 else "decreased"
                explanation_sentences.append(
                    f"Because the condition '{feature}' was met, it {impact} the predicted credit score."
                )
            explanation_str = " ".join(explanation_sentences)
            explanations.append(explanation_str)

        # Compile results
        result = input_df.copy()
        result["Predicted Credit Score"] = predictions.round(2)
        result["Risk Category"] = risk_categories
        result["Decision"] = decisions
        result["Explanation"] = explanations

        # Display result
        print("Final Prediction Results:\n")
        print(result[["Predicted Credit Score", "Risk Category", "Decision", "Explanation"]])

except Exception as e:
    print(f"Error: {str(e)}")


Final Prediction Results:

   Predicted Credit Score Risk Category  \
0                  650.00   Medium Risk   
1                  750.00   Medium Risk   
2                  720.00   Medium Risk   
3                  642.48     High Risk   
4                  780.00      Low Risk   
5                  755.37      Low Risk   
6                  700.00   Medium Risk   
7                  730.00   Medium Risk   
8                  800.00      Low Risk   
9                  600.00     High Risk   

                                   Decision  \
0       Possible Loan Approval with Caution   
1       Possible Loan Approval with Caution   
2       Possible Loan Approval with Caution   
3  Loan Likely Rejected or Needs Collateral   
4                      Likely Loan Approval   
5                      Likely Loan Approval   
6       Possible Loan Approval with Caution   
7       Possible Loan Approval with Caution   
8                      Likely Loan Approval   
9  Loan Likely Rejected or Ne

In [40]:
# STEP 2: Full Flask App Script
import json
from flask import Flask, request, jsonify, render_template_string, redirect, url_for
import pandas as pd
import pickle
from lime.lime_tabular import LimeTabularExplainer
import os

app = Flask(__name__)
# Load model, scaler, encoders
model = pickle.load(open("credit_score_model.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))
label_encoders = pickle.load(open("encoders.pkl", "rb"))

# Create users file if it doesn't exist
if not os.path.exists("users.json"):
    with open("users.json", "w") as f:
        json.dump({}, f)

# HTML Templates
login_page = '''
<!DOCTYPE html>
<html>
<head><title>Login or Signup</title></head>
<body style="font-family:sans-serif;text-align:center;margin-top:10%;">
    <h2>Login or Signup</h2>
    <form method="post" action="/auth">
        <input name="username" placeholder="Username" required><br><br>
        <input name="password" type="password" placeholder="Password" required><br><br>
        <select name="mode">
            <option value="login">Login</option>
            <option value="signup">Sign Up</option>
        </select><br><br>
        <button type="submit">Continue</button>
    </form>
</body>
</html>
'''

upload_page = '''
<!DOCTYPE html>
<html>
<head><title>Credit Score Checker</title></head>
<body style="font-family:sans-serif;text-align:center;margin-top:5%;">
    <h2>Welcome, {{username}}</h2>
    <form action="/check_score" method="post" enctype="multipart/form-data">
        <p>Upload an Excel file (TEST_DATA.xlsx):</p>
        <input type="file" name="file" accept=".xlsx" required><br><br>
        <button type="submit">Check Credit Score</button>
    </form>
</body>
</html>
'''

result_template = '''
<!DOCTYPE html>
<html>
<head><title>Credit Score Result</title></head>
<body style="font-family:sans-serif;margin:5%;">
    <h2>Prediction Result</h2>
    <p><b>Predicted Credit Score:</b> {{score}}</p>
    <p><b>Risk Category:</b> {{risk}}</p>
    <p><b>Decision:</b> {{decision}}</p>
    <p><b>Explanation:</b><br>{{explanation}}</p>
    <a href="/">Go back</a>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(login_page)

@app.route('/auth', methods=['POST'])
def auth():
    username = request.form['username']
    password = request.form['password']
    mode = request.form['mode']

    with open("users.json", "r") as f:
        users = json.load(f)

    if mode == "signup":
        if username in users:
            return "User already exists. Go back and login."
        users[username] = password
        with open("users.json", "w") as f:
            json.dump(users, f)
    elif mode == "login":
        if username not in users or users[username] != password:
            return "Invalid credentials."

    return render_template_string(upload_page, username=username)

@app.route('/check_score', methods=['POST'])
def check_score():
    uploaded_file = request.files['file']
    df = pd.read_excel(uploaded_file)

    required_cols = ['AGE', 'INCOME', 'EMPLOYMENT_STATUS', 'LOAN_AMOUNT', 'LOAN_TERM',
                     'INTEREST_RATE', 'NUM_OF_DEPENDENTS', 'MARITAL_STATUS', 'EDUCATION_LEVEL']

    for col in df.select_dtypes(include=['object']).columns:
        if col in label_encoders:
            le = label_encoders[col]
            df[col] = df[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
        else:
            df[col] = df[col].astype('category').cat.codes

    input_scaled = scaler.transform(df[required_cols])
    score = model.predict(input_scaled)[0]

    if score >= 750:
        risk = "Low Risk"
        decision = "Likely Loan Approval"
    elif score >= 650:
        risk = "Medium Risk"
        decision = "Possible Loan Approval with Caution"
    else:
        risk = "High Risk"
        decision = "Loan Likely Rejected or Needs Collateral"

    explainer = LimeTabularExplainer(training_data=input_scaled,
                                     feature_names=required_cols,
                                     mode="regression")
    exp = explainer.explain_instance(input_scaled[0], model.predict, num_features=5)
    explanation = " ".join([f"Because the condition '{f}' was met, it {'increased' if w > 0 else 'decreased'} the predicted score." for f, w in exp.as_list()])

    return render_template_string(result_template, score=round(score,2), risk=risk, decision=decision, explanation=explanation)

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000, debug=False)



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [39]:
!ngrok http 5000


ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERROR:  Sign up for an account: https://dashboard.ngrok.com/signup
ERROR:  Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_4018
ERROR:  https://ngrok.com/docs/errors/err_ngrok_4018
ERROR:  
